# Creation of New CSV with Languages & no Duplicates

In [ ]:
# !pip install -U git+https://github.com/aboSamoor/polyglot.git@master
# create track language column, done
# df = pd.read_csv('https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv')
# df = df.dropna()
# df = df.drop_duplicates(subset=['track_name', 'artist_name'])
# raw_df = raw_df.drop_duplicates(subset='track_name')
# raw_df.shape
# raw_df.to_csv('spotify_unique_track_id.csv', index=False)
# df.shape

# df.to_csv('spotify_unique_track_id.csv', index=False)
# Data already has duplicate track names and nulls removed.
# from polyglot.text import Text,Word
# def get_language(track_name):
#     try:
#         return(Text(track_name).language.name)
#     except: 
#         return("Unknown")

# df.track_name = df.track_name.astype(str)
# df['track_lang'] = df.track_name.apply(get_language)

# df.to_csv('spotify_unique_track_id_lang.csv', index=False)

# Imports and Installs

In [46]:
import pandas as pd
import numpy as np 
#import pandas_profiling 
from sklearn import preprocessing # for category encoder
# from polyglot.detect import Detector # for language detection stretch
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
# much more efficient for larger files like Nearest Neighbors which the model
import joblib
import plotly.express as px
import plotly.graph_objects as go

# EDA & PROFILE REPORT

In [49]:
df = pd.read_csv('https://raw.githubusercontent.com/Build-Week-Spotify-Song-Suggester-5/Data-Science/master/spotify_unique_track_id_lang.csv')
df.shape

(232725, 19)

In [51]:
# re-order track_lang from the end to after track_name
df = df [['genre', 'artist_name', 'track_name', 'track_lang', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]

In [53]:
# they can only fit 75k on the Heroku
df = df.sort_values('popularity', ascending=False)[:75000]
df.shape

(75000, 19)

# Visualization Stretch Goal Stuff

In [ ]:

# !pip install plotly
# def create_web_chart(df):
#     music_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
    
#     fig = go.Figure()

#     for idx, row in df.iterrows():
#         # vals = list(df.iloc[idx][music_features].values)
#         row_df = pd.DataFrame(df.loc[idx][music_features].T).reset_index().rename(columns={idx:'values', 'index':'feature'})
#         fig.add_trace(go.Scatterpolar(
#                                     r=row_df['values'],
#                                     theta=row_df['feature'],
#                                     fill='toself',
#                                     name= f'{row.artist_name} - {row.track_name}'
#                     ))
#     # fig.show()
#     return fig
# fig = create_web_chart(df.iloc[0:5])
# fig.show()


# first_song_values = list(df.iloc[0][music_features].values)
# second_song_values = list(df.iloc[1][music_features].values)
# fig = px.bar(first_song_df, x='feature', y="values", color='feature')
# fig.show()



# fig.add_trace(go.Scatterpolar(
#       r=second_song_df['values'],
#       theta=second_song_df['feature'],
#       fill='toself',
#       name='Second Song'
# ))

# fig.update_layout(
#   polar=dict(
#     radialaxis=dict(
#       visible=True,
#       range=[0, 1]
#     )),
#   showlegend=True
# )

# fig.show()

# ENCODING OF FEATURES

NOTE THAT IF YOU RUN THIS CELL AGAIN WITHOUT RESTARTING RUNTIME YOU'LL ENCODE AND BIND ANOTHER COPY OF GENRE ENCODINGS SIGNIFICANTLY INCREASING THE SIZE OF THE FEATURE SET WHILE ADDING NEGATIVE VALUE

In [54]:
# helper function to one hot encode genre
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

time_sig_encoding = { '0/4' : 0, '1/4' : 1, 
                         '3/4' : 3, '4/4' : 4,
                         '5/4' : 5}

key_encoding = { 'A' : 0, 'A#' : 1, 'B' : 2,
                    'C' : 3,  'C#' : 4,  'D' : 5,
                    'D#' : 6, 'E' : 7, 'F' : 8,
                    'F#' : 9, 'G' : 10, 'G#' : 11 }

mode_encoding = { 'Major':0, 'Minor':1}      

df['key'] = df['key'].map(key_encoding)
df['time_signature'] = df['time_signature'].map(time_sig_encoding)
df['mode'] = df['mode'].map(mode_encoding)
df = encode_and_bind(df, 'genre')    

In [70]:
df = encode_and_bind(df, 'track_lang')

In [71]:
# df.genre.value_counts()
df.head(2)

,genre,artist_name,track_name,track_lang,track_id,popularity,acousticness,danceability,duration_ms,energy,...,track_lang_Waray,track_lang_Welsh,track_lang_Western Frisian,track_lang_Wolof,track_lang_Xhosa,track_lang_Yoruba,track_lang_Zhuang,track_lang_Zulu,track_lang_un,track_lang_zzp
9027,Dance,Ariana Grande,7 rings,English,14msK75pk3pA33pzPVNtBF,100,0.578,0.725,178640,0.321,...,0,0,0,0,0,0,0,0,0,0
107804,Pop,Ariana Grande,7 rings,English,14msK75pk3pA33pzPVNtBF,100,0.578,0.725,178640,0.321,...,0,0,0,0,0,0,0,0,0,0


In [72]:
df.columns

Index(['genre', 'artist_name', 'track_name', 'track_lang', 'track_id',
       'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       ...
       'track_lang_Waray', 'track_lang_Welsh', 'track_lang_Western Frisian',
       'track_lang_Wolof', 'track_lang_Xhosa', 'track_lang_Yoruba',
       'track_lang_Zhuang', 'track_lang_Zulu', 'track_lang_un',
       'track_lang_zzp'],
      dtype='object', length=160)

# MODELING: Nearest Neighbors
resources: https://scikit-learn.org/stable/modules/neighbors.html

In [73]:
# instantiate model
neigh = NearestNeighbors(n_neighbors=25)
not_features = ['genre', 'artist_name', 'track_name', 'track_lang', 'track_id'] # in case someone wants to change it later
features =[item for item in list(df.columns) if item not in not_features]
X = df[features].values
# y = df[target]
# X.shape 
# y.shape
# target = 'track_id'

%time
neigh.fit(X) # NN doesn't need to fit Y

Wall time: 0 ns


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=25, p=2,
                 radius=1.0)

# Manual Model Test on 1 Song Example

In [74]:
# random test song

test_song = X[8980]
df.iloc[8980]

distance, neighbors = neigh.kneighbors(np.array([test_song]))
distance, neighbors

song_list = []
for idx in neighbors[0][2:]: # this way excludes itself
    row = df.iloc[idx]
    print(f'Artist: {row.artist_name} - Track: {row.track_name}')
    song_list.append((row.artist_name, row.track_name))
print(song_list)

Artist: Clare Dunn - Track: Put You Into Words
Artist: Souly Had - Track: Speaking Of
Artist: Souly Had - Track: Speaking Of
Artist: Chris Brown - Track: High End
Artist: Chris Brown - Track: High End
Artist: Chris Brown - Track: High End
Artist: Feid - Track: Sígueme
Artist: Why Don't We - Track: Invitation
Artist: Why Don't We - Track: Invitation
Artist: Dread Mar I - Track: De Qué Me Vas a Hablar
Artist: NAV - Track: EAT (feat. Gunna)
Artist: NAV - Track: EAT (feat. Gunna)
Artist: The Main Ingredient - Track: Everybody Plays the Fool
Artist: The Main Ingredient - Track: Everybody Plays the Fool
Artist: Easy Life - Track: Temporary Love Part 1
Artist: Sticky Fingers - Track: How To Fly
Artist: Sticky Fingers - Track: How To Fly
Artist: Rush - Track: Fly By Night
Artist: Big K.R.I.T. - Track: Prove It (feat. J. Cole)
Artist: The Rolling Stones - Track: Paint It, Black
Artist: Sigala - Track: Sweet Lovin' - Radio Edit
Artist: One Direction - Track: I Would
Artist: One Direction - Track

# Helper Function to Return 10 Nearest Neighbors

In [64]:
"""
function for what was done manually above
generalized for other dataframes instead of hard coded therefore:
takes in a dataframe, a feature set array, and a song ID
and returns a list of song_id
"""
from typing import List, Tuple
import random 

def closest_ten(df: pd.DataFrame, X_array: np.ndarray ,song_id: int) -> List[Tuple] :
    song = df.iloc[song_id]
    X_song = X[song_id]
    _, neighbors = neigh.kneighbors(np.array([X_song]))
    song_list = []
    for idx in neighbors[0][2:]: 
        row = df.iloc[idx]
       # print(f'Artist: {row.artist_name} - Track: {row.track_name}')
        song_list.append(row.track_id)
    only_10 = random.sample(song_list, 10)
    return(only_10)

In [65]:
# example of function call 
closest_ten(df, X, 9027)

['0c99BUd87HQfzgUUQqRyds',
 '4E8u5JsVnF3qPXbNiR9m25',
 '1depKpsqGCvr8wtjFFaFoO',
 '4E8u5JsVnF3qPXbNiR9m25',
 '3T5mo7mlf8vtRrtEPpOFmY',
 '6B4jeM3B33v2dE709U4dS2',
 '4l5lF04xjhTfYSopCXc4FU',
 '2TXzXJWWzESQq5glbDtZc9',
 '7bhSQqbyUvFVfmXgkF028O',
 '6B4jeM3B33v2dE709U4dS2']

# PCA -> NN

In [32]:
# make copy to transform in case you want to do stuff with the other one later in notebook
df_copy = df[features].copy()

In [33]:
from numpy import array, mean, std, cov
from numpy.linalg import eig
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [34]:
means = df_copy.mean()
# print("\n Means:", means)
scaler = StandardScaler()

In [35]:
pca_X = scaler.fit_transform(df_copy.values)
pca = PCA(0.95)
df_pca = pca.fit(pca_X)

In [36]:
print(pca.explained_variance_)

[4.45794341 2.6619331  1.73667879 1.51142396 1.37838907 1.2611412
 1.13150022 1.10768792 1.07033582 1.06444661 1.06026414 1.05531529
 1.05322221 1.05195636 1.0498224  1.04842724 1.04708279 1.04401482
 1.03562486 1.02767329 1.02411178 1.01842098 1.01476388 1.01108687
 1.00834383 1.00137027 0.98850912 0.92596955 0.82414355 0.79806926
 0.75307744 0.56700277 0.51077434]


In [37]:
%time
pca_neigh = NearestNeighbors(n_neighbors=50)
pca_neigh.fit(pca_X) # NN doesn't need to fit Y

Wall time: 0 ns


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                 radius=1.0)

In [ ]:
# refactor later if have time
# from sklearn.pipeline import make_pipeline
# pipeline = make_pipeline(
#     StandardScaler(),
#     NearestNeighbors())

# pipeline.fit(X)

# Auto Encoder -> NN

In [ ]:
from keras.models import Model

In [ ]:
encoding_dim = 10 # compression of factor 4.1 assuming input of 41

# Export Model with Joblib

In [ ]:
filename = 'NearestNeighborGenres.sav'

In [ ]:
joblib.dump(neigh, filename)

In [ ]:
pca_filename = 'PCANearestNeighborGenres.sav'
joblib.dump(pca_neigh, pca_filename)

In [ ]:
# one hot encoding, rolled back
# def encode_and_bind(original_dataframe, feature_to_encode):
#     dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
#     res = pd.concat([original_dataframe, dummies], axis=1)
#     return(res)

# df = encode_and_bind(df, 'key')
# df = encode_and_bind(df, 'mode')
# df = encode_and_bind(df, 'time_signature')

# Export Model with Pickle

In [ ]:
import pickle

model = neigh

filename = 'filename.pkl'

with open(filename, 'wb') as f:
    pickle.dump(model, f)
#then send this pkl file to DE team

#DE team:
#Make sure to save the pkl file in the same folder as the app

#To read it into the app:

# with open(filename, 'rb') as f:
#     model = pickle.load(f)